# Loading Image

We use this section to load an image and preprocess it. 

In [1]:
# Imports for visualization
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pyautogui

In [2]:
def display_image(img):
    #imS = cv2.resize(img, (1000, 700))  # Resize image
    imS = cv2.resize(img, (800, 800))
    cv2.imshow("image", imS)
    cv2.waitKey(0)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return a

In [3]:
def take_screenshot():
    myScreenshot = pyautogui.screenshot(region=(565,195, 700, 700))
    myScreenshot.save(r'screenshot1.png')
    return r'screenshot1.png'

In [4]:
import tkinter as tk
from tkinter import filedialog

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300)
canvas1.pack()

def takeScreenshot ():
    
    myScreenshot = pyautogui.screenshot()
    file_path = filedialog.asksaveasfilename(defaultextension='.png')
    myScreenshot.save(file_path)

myButton = tk.Button(text='Take Screenshot', command=take_screenshot, bg='green',fg='white',font= 10)
canvas1.create_window(150, 150, window=myButton)

root.mainloop()

In [209]:
#Load image and convert to grayscale
image_path = 'screenshot1.png'
img = cv2.imread(image_path)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Perform Hough transform
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)



In [210]:
vertical_lines = []
horizontal_lines = []
#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if(theta == 0):
        vertical_lines.append(line)



#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if theta != 0.0:        
        horizontal_lines.append(line)
        

In [211]:
def checkMatch(lineset):
    """Checks whether there exists 7 lines of consistent increasing order in set of lines"""
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    for line in linediff:
        # Within 5 px of the other (allowing for minor image errors)
        if np.abs(line - x) < 5:
            cnt += 1
        else:
            cnt = 0
            x = line
    return cnt == 5

In [212]:
len(horizontal_lines)

12

In [213]:
img = cv2.imread(image_path)
for line in vertical_lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [214]:
img = cv2.imread(image_path)
for line in horizontal_lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [215]:

#Convert to numpy arrays. 
horizontal_lines_np = np.concatenate( horizontal_lines, axis=0 )
vertical_lines_np = np.concatenate( vertical_lines, axis=0 )
horizontal_lines_np = np.sort(horizontal_lines_np, axis = 0)
vertical_lines_np = np.sort(vertical_lines_np, axis = 0)

horizontal_lines_np

array([[  1.       ,   1.5707964],
       [ 69.       ,   1.5707964],
       [137.       ,   1.5707964],
       [198.       ,   1.5707964],
       [205.       ,   1.5707964],
       [273.       ,   1.5707964],
       [341.       ,   1.5707964],
       [409.       ,   1.5707964],
       [477.       ,   1.5707964],
       [536.       ,   1.5707964],
       [545.       ,   1.5707964],
       [547.       ,   1.5707964]], dtype=float32)

In [216]:
linediff_hor = np.diff(horizontal_lines_np, axis = 0)
linediff_ver = np.diff(vertical_lines_np, axis = 0)

linediff_hor

array([[68.,  0.],
       [68.,  0.],
       [61.,  0.],
       [ 7.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [59.,  0.],
       [ 9.,  0.],
       [ 2.,  0.]], dtype=float32)

In [217]:
#Remove lines which are very close to each other so that the board detection will be easier. 
def filter_lines(lines, linediff):
    indices_to_delete = []
    for index, diff in enumerate(linediff): 
        if(diff[0] <= 10):
            indices_to_delete.append(index)
    return np.delete(lines, indices_to_delete,0)


In [218]:
filtered_horizontal_lines = filter_lines(horizontal_lines_np, linediff_hor)
filtered_vertical_lines = filter_lines(vertical_lines_np, linediff_ver)

In [219]:
img = cv2.imread(image_path)
for line in filtered_horizontal_lines:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [220]:
#update line differences. 

linediff_hor = np.diff(filtered_horizontal_lines, axis = 0)
linediff_ver = np.diff(filtered_vertical_lines, axis = 0)

In [221]:
linediff_hor

array([[68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [68.,  0.],
       [70.,  0.]], dtype=float32)

In [222]:
def get_chess_lines(lines, linediff):
    indices_to_delete = []
    #find most occuring difference which is probably the width of the chess square. 
    m = stats.mode(linediff)
    mode = m[0][0][0]
    for index, diff in enumerate(linediff): 
        if(abs(diff[0] - mode) >5):
            print(diff[0])
            indices_to_delete.append(index+1)
    return np.delete(lines, indices_to_delete,0)

In [223]:
chess_lines_hor = get_chess_lines(filtered_horizontal_lines, linediff_hor)
chess_lines_ver = get_chess_lines(filtered_vertical_lines, linediff_ver)

In [224]:
img = cv2.imread(image_path)
for line in chess_lines_hor:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    
for line in chess_lines_ver:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

print(chess_lines_hor)
    
display_image(img)

[[  1.          1.5707964]
 [ 69.          1.5707964]
 [137.          1.5707964]
 [205.          1.5707964]
 [273.          1.5707964]
 [341.          1.5707964]
 [409.          1.5707964]
 [477.          1.5707964]
 [547.          1.5707964]]


In [225]:
#Get equations of lines:
def getIntersection(rho1,theta1,rho2, theta2):
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]
    

In [226]:
#Get all intersection points of horizontal and vertical lines to get the corners of the chess squares to extract patches. 

points = []
for i in chess_lines_hor:
    for j in chess_lines_ver:
        rho_hor = i[0]
        theta_hor = i[1]
        rho_ver = j[0]
        theta_ver = j[1]
        points.append(getIntersection(rho_hor, theta_hor, rho_ver, theta_ver)[0])

In [227]:
# Make empty black image
img = cv2.imread(image_path)
for point in points:
    print(point)
    cv2.circle(img, (point[0],point[1]), 5, [0,0,255], thickness=2, lineType=8, shift=0)
#image[10,5]=[0,0,255]

display_image(img)
# # Make one pixel red

# # Save
# cv2.imwrite("result.png",image)

[4, 1]
[72, 1]
[140, 1]
[208, 1]
[276, 1]
[344, 1]
[412, 1]
[480, 1]
[548, 1]
[4, 69]
[72, 69]
[140, 69]
[208, 69]
[276, 69]
[344, 69]
[412, 69]
[480, 69]
[548, 69]
[4, 137]
[72, 137]
[140, 137]
[208, 137]
[276, 137]
[344, 137]
[412, 137]
[480, 137]
[548, 137]
[4, 205]
[72, 205]
[140, 205]
[208, 205]
[276, 205]
[344, 205]
[412, 205]
[480, 205]
[548, 205]
[4, 273]
[72, 273]
[140, 273]
[208, 273]
[276, 273]
[344, 273]
[412, 273]
[480, 273]
[548, 273]
[4, 341]
[72, 341]
[140, 341]
[208, 341]
[276, 341]
[344, 341]
[412, 341]
[480, 341]
[548, 341]
[4, 409]
[72, 409]
[140, 409]
[208, 409]
[276, 409]
[344, 409]
[412, 409]
[480, 409]
[548, 409]
[4, 477]
[72, 477]
[140, 477]
[208, 477]
[276, 477]
[344, 477]
[412, 477]
[480, 477]
[548, 477]
[4, 547]
[72, 547]
[140, 547]
[208, 547]
[276, 547]
[344, 547]
[412, 547]
[480, 547]
[548, 547]


In [228]:
# create a 2D list for easier point arrangement
new_points = []
print(len(points))

if(len(points) != 81) :
    print("Board not detected! Try again!")
else:
    for i in range(0,81):
        if(i % 9) == 0:
            new_points.append([])
        new_points[int(i/9)].append(points[i])

81


In [229]:
new_points

[[[4, 1],
  [72, 1],
  [140, 1],
  [208, 1],
  [276, 1],
  [344, 1],
  [412, 1],
  [480, 1],
  [548, 1]],
 [[4, 69],
  [72, 69],
  [140, 69],
  [208, 69],
  [276, 69],
  [344, 69],
  [412, 69],
  [480, 69],
  [548, 69]],
 [[4, 137],
  [72, 137],
  [140, 137],
  [208, 137],
  [276, 137],
  [344, 137],
  [412, 137],
  [480, 137],
  [548, 137]],
 [[4, 205],
  [72, 205],
  [140, 205],
  [208, 205],
  [276, 205],
  [344, 205],
  [412, 205],
  [480, 205],
  [548, 205]],
 [[4, 273],
  [72, 273],
  [140, 273],
  [208, 273],
  [276, 273],
  [344, 273],
  [412, 273],
  [480, 273],
  [548, 273]],
 [[4, 341],
  [72, 341],
  [140, 341],
  [208, 341],
  [276, 341],
  [344, 341],
  [412, 341],
  [480, 341],
  [548, 341]],
 [[4, 409],
  [72, 409],
  [140, 409],
  [208, 409],
  [276, 409],
  [344, 409],
  [412, 409],
  [480, 409],
  [548, 409]],
 [[4, 477],
  [72, 477],
  [140, 477],
  [208, 477],
  [276, 477],
  [344, 477],
  [412, 477],
  [480, 477],
  [548, 477]],
 [[4, 547],
  [72, 547],
  [140, 54

In [230]:
def get_square_coordinates(points):
    squares = []
    for i in range(0, 8):
        for j in range(0,8):
            squares.append([(i,j), (i, j + 1), (i + 1, j), (i + 1, j + 1)])
    return squares

def get_ith_square(i, points, squares):
    square = squares[i-1]
    return [points[square[0][0]][square[0][1]], points[square[1][0]][square[1][1]],
            points[square[2][0]][square[2][1]],points[square[3][0]][square[3][1]]]

In [231]:
squares = get_square_coordinates(new_points)


In [234]:
img = cv2.imread(image_path)

for i in range(1,65):
    corners = get_ith_square(i, new_points, squares)
    img_patch = img[corners[0][0] : corners[3][0], corners[0][1] : corners[3][1]]
    display_image(cv2.resize(cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY),(28,28)))
